In [ ]:
pip install opencv-python numpy matplotlib scikit-image tensorflow fiftyone kaggle


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import os
import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=[],  # Only images, no labels
    max_samples=5000,
    dataset_name="coco_train_images"
)

In [ ]:
dataset = foz.load_zoo_dataset(
    "coco-2014",
    split="train",
    label_types=[],
    classes=[],
    max_samples=5000,
    dataset_name="coco_train_images2"
)

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prasanshasatpathy/soil-types")

print("Path to dataset files:", path)

In [ ]:
import kagglehub
path = kagglehub.dataset_download("jhislainematchouath/soil-types-dataset")

print("Path to dataset files:", path)

In [ ]:
output_dir = "/home/sala/iit/dsgp/soil_analysis_automation_system/data/processed_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Resize images

In [ ]:
def preprocess_image(image_path, output_path, size=(224, 224)):
    """
    Load an image, resize it, convert to LAB color space, and save.

    Args:
        image_path (str): Path to the input image.
        output_path (str): Path to save the processed image.
        size (tuple): Target dimensions (width, height) for resizing. Default is (224, 224).

    Returns:
        bool: True if the image was processed successfully, False otherwise.
    """
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        return False  # Skip if the image is not found or cannot be loaded

    # Resize the image
    img_resized = cv2.resize(img, size)

    # Convert to LAB color space
    img_lab = cv2.cvtColor(img_resized, cv2.COLOR_BGR2LAB)

    # Save the processed image
    cv2.imwrite(output_path, img_lab)
    return True

In [ ]:
def augment_image(image):
    """
    Augments the image to simulate real-world lighting and focus variations.
    """
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        brightness_range=[0.7, 1.3]
    )

    # Reshape the image for augmentation
    image = image.reshape((1,) + image.shape)  # Adding batch dimension

    # Apply augmentation
    augmented_images = datagen.flow(image, batch_size=1)
    
    # Get the first augmented image
    augmented_image = next(augmented_images)[0].astype(np.uint8)
    
    return augmented_image


In [ ]:
import os
import cv2

def process_images_in_folder(input_folder, output_folder, target_size=(224, 224)):
    """
    Process all images in the input folder and save them to the output folder.

    Args:
        input_folder (str): Path to the folder containing input images.
        output_folder (str): Path to the folder where processed images will be saved.
        target_size (tuple): Target dimensions (width, height) for resizing. Default is (224, 224).
    """
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all images in the folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):  # Case-insensitive check
            # Construct full paths
            image_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"processed_{filename}")

            # Preprocess the image
            success = preprocess_image(image_path, output_path, target_size)
            
            if success:
                print(f"Processed and saved: {output_path}")
            else:
                print(f"Skipped (invalid image): {image_path}")

In [ ]:
input_folder = "/home/sala/soil_data" 
output_folder = '/home/sala/iit/dsgp/preprocessed_data/general_data'  

process_images_in_folder(input_folder, output_folder)